In [27]:
import pandas as pd 
from sqlalchemy import create_engine

# Описание данных

Таблица books
Содержит данные о книгах:

    book_id — идентификатор книги;
    author_id — идентификатор автора;
    title — название книги;
    num_pages — количество страниц;
    publication_date — дата публикации книги;
    publisher_id — идентификатор издателя.

Таблица authors
Содержит данные об авторах:

    author_id — идентификатор автора;
    author — имя автора.

Таблица publishers
Содержит данные об издательствах:

    publisher_id — идентификатор издательства;
    publisher — название издательства;

Таблица ratings
Содержит данные о пользовательских оценках книг:

    rating_id — идентификатор оценки;
    book_id — идентификатор книги;
    username — имя пользователя, оставившего оценку;
    rating — оценка книги.

Таблица reviews
Содержит данные о пользовательских обзорах:

    review_id — идентификатор обзора;
    book_id — идентификатор книги;
    username — имя автора обзора;
    text — текст обзора.

In [28]:
try:
    from paths import bookstore_pass

except ModuleNotFoundError:
    print('Data not for public use, sorry')
    raise

In [29]:
db_config = {'user': 'praktikum_student',
             'pwd': bookstore_pass,
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
             'port': 6432,
             'db': 'data-analyst-final-project-db'} 

connection_string = f"postgresql://{db_config['user']}:{db_config['pwd']}@{db_config['host']}:{db_config['port']}/{db_config['db']}"

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [30]:
sheets = ['books',
         'authors',
         'publishers',
         'ratings',
         'reviews']

In [31]:
for i in sheets:
    query = f'''
    select 
        *
    from 
        {i}
    limit 
        1
    '''
    display(pd.io.sql.read_sql(query, con = engine))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


,author_id,author
0,1,A.S. Byatt


,publisher_id,publisher
0,1,Ace


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


Проверим, что у нас уникальные id книг 

In [32]:
query = '''
select 
    count(distinct(book_id)), 
    title, 
    author_id
from 
    books
group by
    title, 
    author_id
having 
    count(distinct(book_id)) > 1
'''
pd.io.sql.read_sql(query, con = engine) 

,count,title,author_id
0,2,Memoirs of a Geisha,39


Есть один дубль

# Запрсы
##  Сколько книг вышло после 1 января 2000 года?

In [33]:
query = '''
select 
    count(distinct(book_id)) as uniq_ids, 
    count(distinct(title)) as uniq_titles
from 
    books
where 
    publication_date > '2000.01.01'
    
'''
pd.io.sql.read_sql(query, con = engine) 

,uniq_ids,uniq_titles
0,819,818


**Всего вышло 818 уникальных книг**

## Количество обзоров и средняя оценка для каждой книги 

In [34]:
query = '''
select 
    title, 
    author_id, 
    avg(mean_rating) as mean_rating, 
    sum(reviews_count) as reviews_count
from 
    books
left join (
    select 
        book_id, 
        avg(rating) as mean_rating
    from 
        ratings
    group by
        book_id) as ratings on books.book_id = ratings.book_id
left join (
    select 
        book_id, 
        count(text) as reviews_count
    from 
        reviews
    group by
        book_id) as reviews on books.book_id = reviews.book_id

group by
    title, 
    author_id    
'''
pd.io.sql.read_sql(query, con = engine)

,title,author_id,mean_rating,reviews_count
0,The Diamond Age: Or A Young Lady's Illustrate...,429,4.333333,3.0
1,Lover Eternal (Black Dagger Brotherhood #2),239,3.833333,4.0
2,Brother Odd (Odd Thomas #3),127,3.000000,2.0
3,Lisey's Story,542,3.333333,3.0
4,Twenty Love Poems and a Song of Despair,459,4.500000,2.0
...,...,...,...,...
994,The Collectors (Camel Club #2),121,4.000000,2.0
995,Stardust,430,4.000000,4.0
996,Hoot,73,3.333333,3.0
997,Pope Joan,139,4.000000,2.0


**Подсчитали для каждой книги количество обзоров и среднюю оценку**

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [35]:
query = '''
select 
    books.publisher_id,
    publisher, 
    count(title) as books_published
from (
    select 
        *
    from 
        books
    where 
        num_pages > 50
        ) as books
    
left join
    publishers on publishers.publisher_id = books.publisher_id 
group by
    books.publisher_id, 
    publisher
order by 
    books_published desc
limit 
    1
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher,books_published
0,212,Penguin Books,42


**Самое крупное издание - Penguin Books, 42 книги**

## Автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [36]:
query = '''
select 
    mean_rating, 
    author
from (
    select 
        author_id, 
        avg(mean_rating) as mean_rating
    from (
        select 
            book_id, 
            count(rating) as score_count, 
            avg(rating) as mean_rating
        from 
            ratings
        group by
            book_id 
        having 
            count(rating) > 50
            ) as ratings
    left join 
        books on books.book_id = ratings.book_id
    group by 
        author_id
    order by 
        mean_rating desc
    limit 
        1
        ) as ratings
left join 
    authors on authors.author_id = ratings.author_id

'''
pd.io.sql.read_sql(query, con = engine)

,mean_rating,author
0,4.283844,J.K. Rowling/Mary GrandPré


**Роулинг - лучше всех**

## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [37]:
query = '''
select  
    avg(reviews_count)
from (
    select 
        username, 
        count(rating) as score_count
    from 
        ratings
    group by 
        username
    having 
        count(rating) > 50
        ) as ratings
left join (
    select 
        username, 
        count(text) as reviews_count
    from 
        reviews
    group by 
        username) as reviews on reviews.username = ratings.username
'''
pd.io.sql.read_sql(query, con = engine) 

,avg
0,24.333333


**В среднем самые активные юзеры написали по 24 обзора**